In [4]:
import os
import cv2
import numpy as np
import pandas as pd
import onnxruntime
import torch
from scipy.optimize import linear_sum_assignment
import matplotlib.pyplot as plt
import pandas as pd

# TP4

In [14]:
def calculate_iou(box1, box2):
    b1_x1, b1_y1 = box1[0], box1[1]
    b1_x2, b1_y2 = box1[0] + box1[2], box1[1] + box1[3]
    b2_x1, b2_y1 = box2[0], box2[1] 
    b2_x2, b2_y2 = box2[0] + box2[2], box2[1] + box2[3]

    inter_x1 = max(b1_x1, b2_x1)
    inter_y1 = max(b1_y1, b2_y1)
    inter_x2 = min(b1_x2, b2_x2)
    inter_y2 = min(b1_y2, b2_y2)

    inter_area = max(0, inter_x2 - inter_x1) * max(0, inter_y2 - inter_y1)
    
    b1_area = (b1_x2 - b1_x1) * (b1_y2 - b1_y1)
    b2_area = (b2_x2 - b2_x1) * (b2_y2 - b2_y1)
    
    union_area = b1_area + b2_area - inter_area
    
    return inter_area / union_area if union_area > 0 else 0

In [15]:
def calculate_similarity_matrix(tracked_boxes, new_detections):
    # Extraire les bounding boxes des objets Tracker
    tracked_bboxes = [[t.bb_left, t.bb_top, t.bb_width, t.bb_height] for t in tracked_boxes]
    detection_bboxes = [[d.bb_left, d.bb_top, d.bb_width, d.bb_height] for d in new_detections]
    
    # Initialiser la matrice de similarité
    n_tracked = len(tracked_bboxes)
    n_detections = len(detection_bboxes)
    similarity_matrix = np.zeros((n_tracked, n_detections))
    
    # Calculer l'IoU pour chaque paire de boîtes
    for i in range(n_tracked):
        for j in range(n_detections):
            similarity_matrix[i,j] = calculate_iou(tracked_bboxes[i], detection_bboxes[j])
            
    return similarity_matrix


In [16]:
def parse_seqinfo(data_path):
    import configparser
    config = configparser.ConfigParser()
    config.read(os.path.join(data_path, 'seqinfo.ini'))
    return dict(config['Sequence'])

def preprocess_patch(im_crops, width=64, height=128):
    roi_input = cv2.resize(im_crops, (width, height)) 
    roi_input = cv2.cvtColor(roi_input, cv2.COLOR_BGR2RGB)
    std = np.std(roi_input)
    mean = np.mean(roi_input)
    roi_input = (np.asarray(roi_input).astype(np.float32)  -  mean) / std
    roi_input = np.moveaxis(roi_input, -1, 0) 
    object_patch = roi_input.astype('float32') 
    return object_patch


In [17]:
def cosine_similarity(tracked, detected):
    distance_matrix = np.zeros((len(tracked), len(detected)))
    for i, track in enumerate(tracked):
        for j, detect in enumerate(detected):
            distance_matrix[i, j] = np.dot(track.feature, detect.feature) / (np.linalg.norm(track.feature) * np.linalg.norm(detect.feature))
    return distance_matrix

In [18]:
class Tracker:
    def __init__(self, frame, obj_id, bb_left, bb_top, bb_width, bb_height, conf=1, x=-1, y=-1, z=-1):
        self.frame = int(frame)
        self.obj_id = int(obj_id)
        self.bb_left = float(bb_left)
        self.bb_top = float(bb_top)
        self.bb_width = float(bb_width)
        self.bb_height = float(bb_height)
        self.feature = []
        self.frames_perdus = 0
    
    def predict_feature(self, session, frame):
        y_end = int(self.bb_top + self.bb_height)
        x_end = int(self.bb_left + self.bb_width)
        x_start = int(self.bb_left)
        y_start = int(self.bb_top)
        image = frame[y_start:y_end, x_start:x_end]

        # Preprocess the image
        image = preprocess_patch(image)

        with torch.no_grad():
            input_data = torch.from_numpy(image).unsqueeze(0)
            input_data = input_data.numpy()

            input_name = session.get_inputs()[0].name
            feature = session.run(None, {input_name: input_data})[0]
            self.feature = feature[0]
            

In [19]:
# manage the tracking
def track(detected_objects, data_path, model_path):
    
    # video settings
    seq_info = parse_seqinfo(data_path)
    video_path = os.path.join(data_path, 'img1')
    
    # Get sequence length from number of images in img1 folder
    frame_length = len([f for f in os.listdir(video_path) if f.endswith('.jpg')])

    session = onnxruntime.InferenceSession(model_path,providers=onnxruntime.get_available_providers())

    # initialize tracked objects
    tracked_objects = []
    
    # initialize results with pandas frame
    results = pd.DataFrame(columns=['frame', 'id', 'bb_left', 'bb_top', 'bb_width', 'bb_height', 'conf', 'x', 'y', 'z'])

    idx = 1
    all_matrix = []


    THRESHOLD = 0.3
    MAX_LOST_FRAMES = 50 
    ALPHA = 0.4 
    BETA = 0.6
    
    for frame_idx in range(1, frame_length + 1):
        new_tracks = []
        detected_obj_on_frame = detected_objects[detected_objects['frame'] == frame_idx]
        
        if not detected_obj_on_frame.empty:
            frame = cv2.imread(os.path.join(video_path, f'{frame_idx:06d}.jpg'))
            detected_obj_on_frame = [Tracker(obj.frame, obj.id, obj.bb_left, obj.bb_top, 
                                           obj.bb_width, obj.bb_height) 
                                   for _, obj in detected_obj_on_frame.iterrows()]
            
            # Générer les features pour les nouvelles détections
            for obj in detected_obj_on_frame:
                obj.predict_feature(session, frame)
            
            if len(tracked_objects) > 0:
                # Calculer les deux matrices de similarité
                similarity_matrix = calculate_similarity_matrix(tracked_objects, detected_obj_on_frame)
                cosine_similarity_matrix = cosine_similarity(tracked_objects, detected_obj_on_frame)
                
                # Combiner les matrices avec les poids ALPHA et BETA
                cost_matrix = ALPHA * (1 - similarity_matrix) + BETA * (1 - cosine_similarity_matrix)
                
                # Association hongroise
                matches = linear_sum_assignment(cost_matrix)
                
                matched_detections = []
                matched_tracks = []
                
                # Filtrer les associations avec un seuil plus strict
                for track_idx, det_idx in zip(matches[0], matches[1]):
                    iou_score = similarity_matrix[track_idx, det_idx]
                    reid_score = cosine_similarity_matrix[track_idx, det_idx]
                    combined_score = ALPHA * iou_score + BETA * reid_score
                    
                    if combined_score >= THRESHOLD:
                        matched_detections.append(det_idx)
                        matched_tracks.append(track_idx)
                        
                        # Conserver l'ID de la track existante
                        old_track = tracked_objects[track_idx]
                        detected_obj_on_frame[det_idx].obj_id = old_track.obj_id
                        new_tracks.append(detected_obj_on_frame[det_idx])
                
                # Gérer les tracks non appariées
                for i in range(len(tracked_objects)):
                    if i not in matched_tracks:
                        tracked_objects[i].frames_perdus += 1
                        if tracked_objects[i].frames_perdus <= MAX_LOST_FRAMES:
                            # Garder la track en mémoire plus longtemps
                            new_tracks.append(tracked_objects[i])
                
                # Créer de nouvelles tracks uniquement pour les détections vraiment nouvelles
                for i in range(len(detected_obj_on_frame)):
                    if i not in matched_detections:
                        # Vérifier si la détection pourrait correspondre à une track perdue
                        best_match_score = 0
                        best_match_id = None
                        
                        for old_track in tracked_objects:
                            if old_track.frames_perdus > 0:
                                reid_score = np.dot(detected_obj_on_frame[i].feature, old_track.feature) / \
                                           (np.linalg.norm(detected_obj_on_frame[i].feature) * 
                                            np.linalg.norm(old_track.feature))
                                if reid_score > 0.7:  # Seuil élevé pour la réidentification
                                    if reid_score > best_match_score:
                                        best_match_score = reid_score
                                        best_match_id = old_track.obj_id
                        
                        if best_match_id is not None:
                            # Réutiliser l'ancien ID
                            detected_obj_on_frame[i].obj_id = best_match_id
                        else:
                            # Créer un nouveau ID uniquement si nécessaire
                            detected_obj_on_frame[i].obj_id = idx
                            idx += 1
                        new_tracks.append(detected_obj_on_frame[i])
            
            else:
                # Première frame - créer de nouvelles tracks
                for det in detected_obj_on_frame:
                    det.obj_id = idx
                    new_tracks.append(det)
                    idx += 1
        
        # Mettre à jour les tracks
        tracked_objects = new_tracks
        
        # Mettre à jour les résultats
        for track in tracked_objects:
            if track.frames_perdus <= MAX_LOST_FRAMES:
                track_df = pd.DataFrame([{
                    'frame': frame_idx,
                    'id': track.obj_id,
                    'bb_left': track.bb_left,
                    'bb_top': track.bb_top,
                    'bb_width': track.bb_width,
                    'bb_height': track.bb_height,
                    'conf': 1,
                    'x': -1,
                    'y': -1,
                    'z': -1
                }])
                results = pd.concat([results, track_df], ignore_index=True)
    
    return results, all_matrix

In [20]:
def load_detections(det_path):
    # Format attendu : <frame>, <id>, <bb_left>, <bb_top>, <bb_width>, <bb_height>, <conf>, <x>, <y>, <z>
    columns = ['frame', 'id', 'bb_left', 'bb_top', 'bb_width', 'bb_height', 'conf', 'x', 'y', 'z']
    
    # Charger le fichier en utilisant un séparateur d'espace
    detections = pd.read_csv(det_path, header=None, names=columns, sep=' ')
    
    return detections

# Test du tracking
data_path = 'ADL-Rundle-6'  # dossier contenant img1/ et seqinfo.ini
det_path = 'ADL-Rundle-6/det/Yolov5l/det.txt'
model_path = 'reid_osnet_x025_market1501.onnx'

# Charger les détections
detections = load_detections(det_path)

results, all_matrix = track_management(detections, data_path=data_path, model_path=model_path)

2025-01-03 16:50:29.259026 [W:onnxruntime:, coreml_execution_provider.cc:115 GetCapability] CoreMLExecutionProvider::GetCapability, number of partitions supported by CoreML: 3 number of nodes in the graph: 391 number of nodes supported by CoreML: 389


In [21]:
def display_tracking_results(results, data_path, output_path):
    # parse seqinfo
    seq_info = parse_seqinfo(data_path)

    video_path = os.path.join(data_path, 'img1')

    # Configure video reader
    # Get first frame to determine dimensions
    first_frame = cv2.imread(os.path.join(video_path, '000001.jpg'))
    frame_height, frame_width = first_frame.shape[:2]
    
    # Get number of frames from directory
    frame_length = len([f for f in os.listdir(video_path) if f.endswith('.jpg')])
    fps = 30  # Default fps if not available
    
    if seq_info is not None:
        # Use seq_info values if available
        if 'imWidth' in seq_info:
            frame_width = int(seq_info['imWidth'])
        if 'imHeight' in seq_info:
            frame_height = int(seq_info['imHeight']) 
        if 'seqLength' in seq_info:
            frame_length = int(seq_info['seqLength'])
        if 'frameRate' in seq_info:
            fps = int(seq_info['frameRate'])
    
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_path, fourcc, fps, (frame_width, frame_height))    

    # draw bounding boxes
    for frame_idx in range(1, frame_length + 1):
        frame = cv2.imread(os.path.join(video_path, f'{frame_idx:06d}.jpg'))
        tracks = results[results['frame'] == frame_idx]
        for track in tracks.values:
            x, y, w, h = track[2:6]
            cv2.rectangle(frame, (int(x), int(y)), (int(x + w), int(y + h)), (0, 255, 0), 2)
            cv2.putText(frame, str(int(track[1])), (int(x), int(y) - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)

        cv2.imshow('frame', frame)
        out.write(frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    
    out.release()
    cv2.destroyAllWindows()


In [22]:
output_path = 'tp4_result.mp4'
display_tracking_results(results, data_path, output_path)

# TP5

In [10]:
from ultralytics import YOLO
import numpy as np
import cv2
import os
import pandas as pd
from tqdm import tqdm

def generate_detections(video_path, output_path, conf_threshold=0.5):  
    model = YOLO('yolov8s.pt') 
    
    # Préparer le fichier de sortie
    detections = []
    
    # Liste toutes les images dans le dossier
    images = sorted([f for f in os.listdir(video_path) if f.endswith('.jpg')])
    
    # Traiter chaque image
    for frame_id, img_name in tqdm(enumerate(images), desc="Detecting", total=len(images)):
        
        img_path = os.path.join(video_path, img_name)
        
        # Faire la détection
        results = model(img_path)[0]
        
        # Filtrer uniquement les détections de personnes (classe 0 dans COCO)
        for r in results.boxes.data.tolist():
            x1, y1, x2, y2, conf, class_id = r
            
            if class_id == 0 and conf > conf_threshold:  
                
                w = x2 - x1
                h = y2 - y1
                
                
                detection = [
                    frame_id + 1, 
                    -1,  
                    x1, 
                    y1,
                    w,   
                    h,     
                    conf, 
                    -1, -1, -1  
                ]
                detections.append(detection)
    
    # Sauvegarder les détections
    det_array = np.array(detections)
    np.savetxt(
        output_path,
        det_array,
        fmt=['%d', '%d', '%.2f', '%.2f', '%.2f', '%.2f', '%.6f', '%d', '%d', '%d'],
        delimiter=' '
    )
    
    return len(detections)


# Chemins
video_path = "ADL-Rundle-6/img1"
output_path = "ADL-Rundle-6/det/yolov8n/det.txt"
    
# Créer le dossier de sortie si nécessaire
os.makedirs(os.path.dirname(output_path), exist_ok=True)
    
# Générer les détections
nb_detections = generate_detections(
        video_path=video_path,
        output_path=output_path,
        conf_threshold=0.5
)

detections = pd.read_csv(output_path, header=None, sep=' ')


Detecting:   0%|          | 0/525 [00:00<?, ?it/s]

/opt/homebrew/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


image 1/1 /Users/gaspardsaliou/Documents/epita/ing3/visualObjectTracking/tp2-3/ADL-Rundle-6/img1/000001.jpg: 384x640 9 persons, 1 backpack, 1 tv, 94.1ms
Speed: 3.0ms preprocess, 94.1ms inference, 6.4ms postprocess per image at shape (1, 3, 384, 640)


Detecting:   0%|          | 1/525 [00:01<09:27,  1.08s/it]


image 1/1 /Users/gaspardsaliou/Documents/epita/ing3/visualObjectTracking/tp2-3/ADL-Rundle-6/img1/000002.jpg: 384x640 7 persons, 1 backpack, 2 handbags, 1 tv, 57.0ms
Speed: 1.4ms preprocess, 57.0ms inference, 0.4ms postprocess per image at shape (1, 3, 384, 640)

image 1/1 /Users/gaspardsaliou/Documents/epita/ing3/visualObjectTracking/tp2-3/ADL-Rundle-6/img1/000003.jpg: 384x640 7 persons, 1 backpack, 2 handbags, 1 tv, 69.9ms
Speed: 1.2ms preprocess, 69.9ms inference, 0.5ms postprocess per image at shape (1, 3, 384, 640)


...


image 1/1 /Users/gaspardsaliou/Documents/epita/ing3/visualObjectTracking/tp2-3/ADL-Rundle-6/img1/000523.jpg: 384x640 10 persons, 1 skateboard, 78.7ms
Speed: 1.5ms preprocess, 78.7ms inference, 0.5ms postprocess per image at shape (1, 3, 384, 640)

image 1/1 /Users/gaspardsaliou/Documents/epita/ing3/visualObjectTracking/tp2-3/ADL-Rundle-6/img1/000524.jpg: 384x640 9 persons, 76.4ms
Speed: 1.3ms preprocess, 76.4ms inference, 0.5ms postprocess per image at shape (1, 3, 384, 640)


Detecting: 100%|█████████▉| 524/525 [00:46<00:00, 11.26it/s]


image 1/1 /Users/gaspardsaliou/Documents/epita/ing3/visualObjectTracking/tp2-3/ADL-Rundle-6/img1/000525.jpg: 384x640 9 persons, 65.4ms
Speed: 1.4ms preprocess, 65.4ms inference, 0.3ms postprocess per image at shape (1, 3, 384, 640)


Detecting: 100%|██████████| 525/525 [00:46<00:00, 11.34it/s]

Détections générées : 4227

Premières détections :
   0  1        2       3       4       5         6  7  8  9
0  1 -1     0.43  341.42  117.67  561.82  0.844173 -1 -1 -1
1  1 -1  1700.27  390.84  160.26  337.91  0.839954 -1 -1 -1
2  1 -1   256.67  457.55   98.77  245.05  0.771513 -1 -1 -1
3  1 -1   123.84  505.35   87.59  237.99  0.670329 -1 -1 -1
4  1 -1  1286.16  463.12   71.04  194.19  0.651424 -1 -1 -1


In [11]:
# Test du tracking
data_path = 'ADL-Rundle-6'  # dossier contenant img1/ et seqinfo.ini
det_path = 'ADL-Rundle-6/det/yolov8n/det.txt'
model_path = 'reid_osnet_x025_market1501.onnx'

# Charger les détections
detections = load_detections(det_path)

results, all_matrix = track(detections, data_path, model_path)

2025-01-03 15:03:19.430600 [W:onnxruntime:, coreml_execution_provider.cc:115 GetCapability] CoreMLExecutionProvider::GetCapability, number of partitions supported by CoreML: 3 number of nodes in the graph: 391 number of nodes supported by CoreML: 389


In [23]:
output_path = 'tp5_result.mp4'
display_tracking_results(results, data_path, output_path)